# Marketing Data Foundation Setup Script

## Project Setup using Notebook

### *Pre-requisites:*

1. Role with access to create Compute Pools, create Warehouses and Databases.
2. Any environment to run notebooks like VS code, Jupyterlab, etc.
3. Have Docker Desktop installed.
4. Install and configure the Snow CLI to deploy the application in your account.


### Config files
The project contains some config file that should be updated with your own environment variables.
1. [app.config.json](app.config.json)

## Setup local environment

In [34]:
! python3.9 -m venv .venv 
! source .venv/bin/activate 
! pip install -r requirements.txt

### Load App Configurations

In [35]:
from scripts.app_config import get_app_config

app_conf_file = 'app.config.json'
app_config = get_app_config(app_conf_file)

print(f'Success')

Image Repository: marketing-data-foundation:latest
Success


## Create Role


In [36]:
from scripts.auth import get_conn_obj
from scripts.executeStatement import executeStatement
import os

user = os.environ['USER']
role = f"{app_config['database']}_ROLE"
print(executeStatement(f"CREATE ROLE IF NOT EXISTS {role};"))
print(executeStatement(f"GRANT CREATE COMPUTE POOL ON ACCOUNT TO ROLE {role};"))
print(executeStatement(f"GRANT BIND SERVICE ENDPOINT ON ACCOUNT TO ROLE {role};"))
print(executeStatement(f"GRANT CREATE WAREHOUSE ON ACCOUNT TO ROLE {role};"))
print('Success')


CREATE ROLE IF NOT EXISTS MARKETING_DATA_FOUNDATION_V3_ROLE;
+--------+
| status |
|--------|
+------------------------------------------------------------------------+
| status                                                                 |
|------------------------------------------------------------------------|
| MARKETING_DATA_FOUNDATION_V3_ROLE already exists, statement succeeded. |
+------------------------------------------------------------------------+


CREATE ROLE IF NOT EXISTS MARKETING_DATA_FOUNDATION_V3_ROLE;
+--------+
| status |
|--------|
+------------------------------------------------------------------------+
| status                                                                 |
|------------------------------------------------------------------------|
| MARKETING_DATA_FOUNDATION_V3_ROLE already exists, statement succeeded. |
+------------------------------------------------------------------------+


GRANT CREATE COMPUTE POOL ON ACCOUNT TO ROLE MARKETING_DAT

### Replace application configuration files keys

In [37]:
import os

from scripts.update_file_variables import file_replace

app_config_f = 'app/src/manifest.yml'
fullstack_config_f = 'app/src/fullstack.yaml'
makefile_f = 'Makefile'

replace_map = {
    "<image_repository_path>": os.environ["IMAGE_REPOSITORY"],
    "<image_repo_short_path>": os.environ["IMAGE_REPO_SHORT"],
    "<role>": role
}

file_replace(app_config_f, replace_map)
file_replace(fullstack_config_f, replace_map)
file_replace(makefile_f, replace_map)
print(f'Success')

Success


# Clear resources (optional)

In [38]:
! cd app && snow app teardown --force

Role ACCOUNTADMIN does not own any application object with the name Marketing_Data_Foundation_Starter_V3_jbellegarde, or the application object does not exist.
Dropping application package Marketing_Data_Foundation_Starter_V3_pkg_jbellegarde now.
Dropped application package Marketing_Data_Foundation_Starter_V3_pkg_jbellegarde successfully.
Teardown is now complete.


In [39]:
# print(sp_session.sql(f"USE DATABASE {app_config['database']}"))
from scripts.executeStatement import executeStatement

print(executeStatement(f"ALTER COMPUTE POOL IF EXISTS {app_config['compute_pool']} STOP ALL;"))
print(executeStatement(f"DROP SERVICE IF EXISTS {app_config['container_service']}"))
print(executeStatement(f"DROP COMPUTE POOL IF EXISTS {app_config['compute_pool']}"))
print(executeStatement(f"DROP IMAGE REPOSITORY IF EXISTS {app_config['image_stage']};"))
print(executeStatement(f"DROP DATABASE IF EXISTS {app_config['database']};"))
print(executeStatement(f"DROP WAREHOUSE IF EXISTS {app_config['warehouse']};"))


ALTER COMPUTE POOL IF EXISTS MARKETING_DATA_FOUNDATION_COMPUTE_POOL STOP ALL;
+--------+
| status |
|--------|
+----------------------------------+
| status                           |
|----------------------------------|
| Statement executed successfully. |
+----------------------------------+


ALTER COMPUTE POOL IF EXISTS MARKETING_DATA_FOUNDATION_COMPUTE_POOL STOP ALL;
+--------+
| status |
|--------|
+----------------------------------+
| status                           |
|----------------------------------|
| Statement executed successfully. |
+----------------------------------+


DROP SERVICE IF EXISTS MARKETING_DATA_FOUNDATION_SERVICE
+--------+
| status |
|--------|
+------------------------------------------------------------------------------+
| status                                                                       |
|------------------------------------------------------------------------------|
| Drop statement executed successfully (MARKETING_DATA_FOUNDATION_SERVI

## Setup Snowflake
### Create database, schema and stages

In [40]:
create_app_package = f"CREATE DATABASE IF NOT EXISTS {app_config['database']};"
create_dedicated_warehouse = f"CREATE WAREHOUSE IF NOT EXISTS {app_config['warehouse']} WITH WAREHOUSE_SIZE = 'XSMALL' AUTO_SUSPEND = 60 AUTO_RESUME = TRUE INITIALLY_SUSPENDED = TRUE;"
grant_database = f"GRANT ALL PRIVILEGES ON DATABASE {app_config['database']} TO ROLE {role}"



create_app_schema = f"CREATE SCHEMA IF NOT EXISTS {app_config['database']}.{app_config['schema']};"

grant_schema = f"GRANT ALL PRIVILEGES ON SCHEMA {app_config['database']}.{app_config['schema']} TO ROLE {role};"

grant_tables = f"GRANT SELECT ON ALL TABLES IN SCHEMA {app_config['database']}.{app_config['schema']} TO ROLE {role};"

create_image_repo = f"CREATE IMAGE REPOSITORY IF NOT EXISTS {app_config['database']}.{app_config['schema']}.{app_config['image_stage']};"

print(executeStatement(create_app_package))
print(executeStatement(create_app_schema))
print(executeStatement(create_image_repo))
print(executeStatement(grant_database))
print(executeStatement(grant_schema))
print(executeStatement(grant_tables))
print(executeStatement(create_dedicated_warehouse))

CREATE DATABASE IF NOT EXISTS MARKETING_DATA_FOUNDATION_V3;
+--------+
| status |
|--------|
+-------------------------------------------------------------+
| status                                                      |
|-------------------------------------------------------------|
| Database MARKETING_DATA_FOUNDATION_V3 successfully created. |
+-------------------------------------------------------------+


CREATE DATABASE IF NOT EXISTS MARKETING_DATA_FOUNDATION_V3;
+--------+
| status |
|--------|
+-------------------------------------------------------------+
| status                                                      |
|-------------------------------------------------------------|
| Database MARKETING_DATA_FOUNDATION_V3 successfully created. |
+-------------------------------------------------------------+


CREATE SCHEMA IF NOT EXISTS MARKETING_DATA_FOUNDATION_V3.METADATA;
+--------+
| status |
|--------|
+---------------------------------------+
| status                    

### Upload Sample Data

In [41]:
create_sample_db = f"CREATE DATABASE IF NOT EXISTS {app_config['sample_db']};"
create_sample_schema = f"CREATE SCHEMA IF NOT EXISTS {app_config['sample_db']}.{app_config['sample_schema']};"
create_sample_stage = f"CREATE STAGE IF NOT EXISTS {app_config['sample_db']}.{app_config['sample_schema']}.{app_config['sample_stage']};"

print(executeStatement(create_sample_db))
print(executeStatement(create_sample_schema))
print(executeStatement(create_sample_stage))

CREATE DATABASE IF NOT EXISTS C360_SAMPLE_DB;
+--------+
| status |
|--------|
+-----------------------------------------------------+
| status                                              |
|-----------------------------------------------------|
| C360_SAMPLE_DB already exists, statement succeeded. |
+-----------------------------------------------------+


CREATE DATABASE IF NOT EXISTS C360_SAMPLE_DB;
+--------+
| status |
|--------|
+-----------------------------------------------------+
| status                                              |
|-----------------------------------------------------|
| C360_SAMPLE_DB already exists, statement succeeded. |
+-----------------------------------------------------+


CREATE SCHEMA IF NOT EXISTS C360_SAMPLE_DB.C360_SAMPLE_SCHEMA;
+--------+
| status |
|--------|
+---------------------------------------------------------+
| status                                                  |
|---------------------------------------------------------|
| 

In [42]:
from scripts.executeStatement import executeCopyToStage
sample_stage= f"@{app_config['sample_db']}.{app_config['sample_schema']}.{app_config['sample_stage']}"
ga_data = "data/ga_data/"
sf_data = "data/sf_data/"
worldcities = "data/worldcities.csv"

print(executeCopyToStage(ga_data,f"{sample_stage}/data/ga_data/"))
print(executeCopyToStage(sf_data, f"{sample_stage}/data/sf_data/"))
print(executeCopyToStage(worldcities, f"{sample_stage}/data"))

+------------------------------------------------------------------------------+
|        |        |         |          | source_ | target_c |        |         |
|        |        | source_ | target_s | compres | ompressi |        |         |
| source | target | size    | ize      | sion    | on       | status | message |
|--------+--------+---------+----------+---------+----------+--------+---------|
+------------------------------------------------------------------------------+
|         |         |         |         | source_ | target_ |         |        |
|         |         | source_ | target_ | compres | compres |         | messag |
| source  | target  | size    | size    | sion    | sion    | status  | e      |
|---------+---------+---------+---------+---------+---------+---------+--------|
| ga_samp | ga_samp | 2502215 | 0       | GZIP    | GZIP    | SKIPPED |        |
| le_data | le_data | 3       |         |         |         |         |        |
| _0_0_0. | _0_0_0. |       

In [43]:
from scripts.update_file_variables import file_replace

data_script_f = 'scripts/build_raw_samples.sql'

replace_map = {
    "<DB>": app_config['sample_db'],
    "<SCHEMA>": app_config['sample_schema'],
    "<STAGE>": app_config['sample_stage']
}

file_replace(data_script_f, replace_map)
print(f'Success')

Success


In [44]:
! snow sql -f scripts/build_raw_samples.sql

create or replace TABLE C360_SAMPLE_DB.C360_SAMPLE_SCHEMA.SIMPLEMAPS (
        CITY VARCHAR(16777216),
        CITY_ASCII VARCHAR(16777216),
        LAT FLOAT,
        LNG FLOAT,
        COUNTRY VARCHAR(16777216),
        ISO2 VARCHAR(16777216),
        ISO3 VARCHAR(16777216),
        ADMIN_NAME VARCHAR(16777216),
        CAPITAL VARCHAR(16777216),
        POPULATION VARCHAR(16777216),
        ID NUMBER(38,0));
+--------+
| status |
|--------|
+----------------------------------------+
| status                                 |
|----------------------------------------|
| Table SIMPLEMAPS successfully created. |
+----------------------------------------+

CREATE OR REPLACE FILE FORMAT C360_SAMPLE_DB.C360_SAMPLE_SCHEMA.CSV_FORMAT TYPE=CSV
    SKIP_HEADER=1
    FIELD_DELIMITER=','
    TRIM_SPACE=TRUE
    FIELD_OPTIONALLY_ENCLOSED_BY='"'
    REPLACE_INVALID_CHARACTERS=TRUE
    DATE_FORMAT=AUTO
    TIME_FORMAT=AUTO
    TIMESTAMP_FORMAT=AUTOs;
+--------+
| status |
|--------|
+-------------

### Build and Upload Docker Images

In [45]:
! make all

snow spcs image-registry login
Login Succeeded
cd backend && docker build --platform linux/amd64 -t eap_backend . && cd ..
[+] Building 0.0s (0/0)  docker:desktop-linux
[+] Building 0.0s (0/1)                                    docker:desktop-linux
[+] Building 0.2s (2/3)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 518B                                       0.0s
 => [internal] load metadata for docker.io/library/python:3.8              0.2s
 => [auth] library/python:pull token for registry-1.docker.io              0.0s
[+] Building 0.4s (2/3)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 518B                                       0.0s
 => [internal] load metadata for docker.io/library/python:3.8              0.4s
 => [auth] library/python:pull 

### Deploy Native Application

In [46]:
! cd app && snow app run

Creating new application package Marketing_Data_Foundation_Starter_V3_pkg_jbellegarde in account.
Checking if stage Marketing_Data_Foundation_Starter_V3_pkg_jbellegarde.app_src.stage exists, or creating a new one if none exists.
Performing a diff between the Snowflake stage and your local deploy_root ('/Users/jbellegarde/innovation_showcase_team/quickstarts/sfguide-marketing-data-foundation-starter-v3/scripts/app/output/deploy') directory.
Local changes to be deployed:
  added:    src/fullstack.yaml -> fullstack.yaml
  added:    src/manifest.yml -> manifest.yml
  added:    src/readme.md -> readme.md
  added:    src/setup.sql -> setup.sql
Updating the Snowflake stage from your local /Users/jbellegarde/innovation_showcase_team/quickstarts/sfguide-marketing-data-foundation-starter-v3/scripts/app/output/deploy directory.
Validating Snowflake Native App setup script.
Creating new application object Marketing_Data_Foundation_Starter_V3_jbellegarde in account.
Application 'MARKETING_DATA_FOUN

### Create Compute Pool

In [47]:
import os
import subprocess
appName = f"MARKETING_DATA_FOUNDATION_STARTER_V3_{os.environ['USER'].upper()}"
print(appName)

MARKETING_DATA_FOUNDATION_STARTER_V3_JBELLEGARDE


### Load Predefined Data Models

In [48]:
from scripts.load_models import load_models

models_folder = "backend/predefined_models"

load_models(appName, models_folder)

INSERT INTO MARKETING_DATA_FOUNDATION_STARTER_V3_JBELLEGARDE.METADATA.MODELS(MODEL_ID, MODEL_NAME, TARGET_DATABASE, TARGET_SCHEMA, MODEL_UI, CREATED_TIMESTAMP)
                            SELECT 2, 'StandardizeModelFacebookFivetran', 'MARKETING_DATA_FOUNDATION_STARTER_V3_JBELLEGARDE', 'TARGET', parse_json($${'databases': [{'databaseName': 'MARKETING_DATA_FOUNDATION_STARTER_V3_JBELLEGARDE', 'schemas': [{'databaseName': 'MARKETING_DATA_FOUNDATION_STARTER_V3_JBELLEGARDE', 'schemaName': 'TARGET', 'tables': [{'alias': 'DIM10', 'columns': [{'columnName': 'ID', 'object': 'MARKETING_DATA_FOUNDATION_STARTER_V3_JBELLEGARDE.TARGET.DIM_CAMPAIGN_FIVETRAN_FACEBOOK', 'sqlType': 'NUMBER', 'type': 'NUMBER'}, {'columnName': 'CAMPAIGN_NAME', 'object': 'MARKETING_DATA_FOUNDATION_STARTER_V3_JBELLEGARDE.TARGET.DIM_CAMPAIGN_FIVETRAN_FACEBOOK', 'sqlType': 'VARCHAR', 'type': 'VARCHAR'}, {'columnName': 'ACCOUNT_ID', 'object': 'MARKETING_DATA_FOUNDATION_STARTER_V3_JBELLEGARDE.TARGET.DIM_CAMPAIGN_FIVETRAN_FACEBOO

In [49]:
from scripts.executeStatement import executeStatement
create_compute_pool_sql = f"CREATE COMPUTE POOL IF NOT EXISTS {app_config['compute_pool']} for application {appName}\
    MIN_NODES = 1 \
    MAX_NODES = 1 \
    AUTO_SUSPEND_SECS = 120 \
    INSTANCE_FAMILY = CPU_X64_S;"
grant_usage_sql = f"GRANT USAGE, MONITOR ON COMPUTE POOL {app_config['compute_pool']} \
    TO application {appName};"

grant_usage_wh = f"GRANT USAGE, MONITOR ON WAREHOUSE {app_config['warehouse']} \
    TO application {appName};"

grant_bind_service = f"GRANT BIND SERVICE ENDPOINT ON ACCOUNT TO application {appName};"

print(executeStatement(create_compute_pool_sql))
print(executeStatement(grant_usage_sql))
print(executeStatement(grant_usage_wh))
print(executeStatement(grant_bind_service))



CREATE COMPUTE POOL IF NOT EXISTS MARKETING_DATA_FOUNDATION_COMPUTE_POOL for application MARKETING_DATA_FOUNDATION_STARTER_V3_JBELLEGARDE    MIN_NODES = 1     MAX_NODES = 1     AUTO_SUSPEND_SECS = 120     INSTANCE_FAMILY = CPU_X64_S;
+--------+
| status |
|--------|
+------------------------------------------------------------------------------+
| status                                                                       |
|------------------------------------------------------------------------------|
| Compute pool MARKETING_DATA_FOUNDATION_COMPUTE_POOL successfully created.    |
| Please run 'DESCRIBE COMPUTE POOL MARKETING_DATA_FOUNDATION_COMPUTE_POOL' to |
| check the compute pool state. NOTE the compute pool is not ready to deploy a |
| service or job before reaching ACTIVE or IDLE state.                         |
+------------------------------------------------------------------------------+


CREATE COMPUTE POOL IF NOT EXISTS MARKETING_DATA_FOUNDATION_COMPUTE_POOL for applic

#### Upload Semantic Model Configuration

In [50]:
from scripts.upload_files import upload_files_stage
from scripts.update_file_variables import file_replace

database = appName
schema = 'LLM'
semantic_models_stage = 'SEMANTIC_MODEL'
llm_config_stage = 'CONFIGURATION'

replace_map = {
    "<target_database>": appName
}
file_replace('./assistant/semantic_models/UnifiedMarketingModel_CAMPAIGN_PERF.yaml', replace_map)

subprocess.run(['snow', 'stage', 'copy', './assistant/config/assistant_config.yaml', f"@{database}.{schema}.{llm_config_stage}"])
subprocess.run(['snow', 'stage', 'copy', './assistant/semantic_models/UnifiedMarketingModel_CAMPAIGN_PERF.yaml', f"@{database}.{schema}.{semantic_models_stage}"])

print('Success')

NameError: name 'subprocess' is not defined

In [ ]:
grant_sample_db = f"GRANT USAGE ON DATABASE {app_config['sample_db']} TO APPLICATION {appName};"
grant_sample_schema = f"GRANT USAGE ON SCHEMA {app_config['sample_db']}.{app_config['sample_schema']} TO APPLICATION {appName};"
grant_sample_select = f"GRANT SELECT ON ALL TABLES IN SCHEMA {app_config['sample_db']}.{app_config['sample_schema']} TO APPLICATION  {appName};"

print(executeStatement(grant_sample_db))
print(executeStatement(grant_sample_schema))
print(executeStatement(grant_sample_select))


## Start container service

In [ ]:
service_query = f"call {appName}.app_public.start_app(\'{app_config['compute_pool']}\',\'{app_config['warehouse']}\')"
executeStatement(service_query)

### Show container endpoint

In [ ]:
from scripts.executeStatement import executeStatement
from scripts.endpoint_provider import get_public_url_na
get_public_url_na(appName, executeStatement)